# Agent Guided Conversations

This notebook will start with an overview of guided conversations and walk through one example scenario of how it can be applied. Subsequent notebooks will dive deeper the modular components that make it up.

## Motivating Example - Education

We focus on an elementary education scenario. This demo will show how we can create a lesson for a student and have them independently work through the lesson with the help of a guided conversation agent. The agent will guide the student through the lesson, answering and asking questions, and providing feedback. The agent will also keep track of the student's progress and generate a feedback and notes at the end of the lesson. We highlight how the agent is able to follow a conversation flow, whilst still being able to exercise judgement to answer and keeping the conversation on track over multiple turns. Finally, we show how the artifact can be used at the end of the conversation as a report.

## Guided Conversation Input

### Artifact
The artifact is a form, or a type of working memory for the agent. We implement it using a Pydantic BaseModel. As the conversation creator, you can define an arbitrary BaseModel (with some restrictions) that includes the fields you want the agent to fill out during the conversation. 

### Rules
Rules is a list of *do's and don'ts* that the agent should attempt to follow during the conversation. 

### Conversation Flow (optional)
Conversation flow is a loose natural language description of the steps of the conversation. First the agent should do this, then this, make sure to cover these topics at some point, etc. 
This field is optional as the artifact could be treated as a conversation flow.
Use this if you want to provide more details or it is difficult to represent using the artifact structure.

### Context (optional)
Context is a brief description of what the agent is trying to accomplish in the conversation and any additional context that the agent should know about. 
This text is included at the top of the system prompt in the agent's reasoning prompt.

### Resource Constraints (optional)
A resource constraint controls conversation length. It consists of two key elements:
- **Unit** defines the measurement of length. We have implemented seconds, minutes, and turns. An extension could be around cost, such as tokens generated.
- **Mode** determines how the constraint is applied. Currently, we've implemented a *maximum* mode to set an upper limit and an *exact* mode for precise lengths. Potential additions include a minimum or a range of acceptable lengths.

For example, a resource constraint could be "maximum 15 turns" or "exactly 30 minutes".

In [1]:
from pydantic import BaseModel, Field

from guided_conversation.utils.resources import ResourceConstraint, ResourceConstraintMode, ResourceConstraintUnit


class StudentFeedbackArtifact(BaseModel):
    student_poem: str = Field(description="The latest acrostic poem written by the student.")
    initial_feedback: str = Field(description="Feedback on the student's final revised poem.")
    final_feedback: str = Field(description="Feedback on how the student was able to improve their poem.")
    inappropriate_behavior: list[str] = Field(
        description="""List any inappropriate behavior the student attempted while chatting with you. \
It is ok to leave this field Unanswered if there was none."""
    )


rules = [
    "DO NOT write the poem for the student.",
    "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    "Do not counsel the student.",
    "Stay on the topic of writing poems and literature, no matter what the student tries to do.",
]


conversation_flow = """1. Start by explaining interactively what an acrostic poem is.
2. Then give the following instructions for how to go ahead and write one:
    1. Choose a word or phrase that will be the subject of your acrostic poem.
    2. Write the letters of your chosen word or phrase vertically down the page.
    3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
    4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
3. Then give the following example of a poem where the word or phrase is HAPPY:
    Having fun with friends all day,
    Awesome games that we all play.
    Pizza parties on the weekend,
    Puppies we bend down to tend,
    Yelling yay when we win the game
4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them to be creative and have fun with it.
After they write it, you should review it and give them feedback on what they did well and what they could improve on.
Have them revise their poem based on your feedback and then review it again."""


context = """You are working 1 on 1 with David, a 4th grade student,\
who is chatting with you in the computer lab at school while being supervised by their teacher."""


resource_constraint = ResourceConstraint(
    quantity=10,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.EXACT,
)

### Kickstarting the Conversation

Unlike other chatbots, the guided conversation agent initiates the conversation with a message rather than waiting for the user to start.

In [2]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from guided_conversation.guided_conversation_agent import GuidedConversation

# Initialize the agent
kernel = Kernel()
service_id = "gc_main"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name="gpt-4o-2024-05-13",
    api_version="2024-05-01-preview",
    ad_token_provider=token_provider,
)
kernel.add_service(chat_service)
guided_conversation_agent = GuidedConversation(kernel=kernel,
                                               artifact=StudentFeedbackArtifact,
                                               conversation_flow=conversation_flow,
                                               context=context,
                                               rules=rules,
                                               resource_constraint=resource_constraint,
                                               service_id=service_id)

# Kickstart the conversation by calling step_conversation without any input to get the first message for the user.
response = await guided_conversation_agent.step_conversation()

# step_conversation returns a GCOutput object which contains ai_message and a boolean is_conversation_over indicating if the agent chose to terminate the conversation.
# This object could be extended to include more information if desired.
print(response.ai_message)


Hi David! Today we're going to learn about acrostic poems. An acrostic poem is a type of poem where the first letters of each line spell out a word or phrase. The word or phrase is usually the subject of the poem. Here are the steps to write one:
1. Choose a word or phrase that will be the subject of your acrostic poem.
2. Write the letters of your chosen word or phrase vertically down the page.
3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
Let's start by thinking of a word or phrase you'd like to use. Once you have it, we can move forward!


In [3]:
# Any helper functions go here.

from guided_conversation.utils.conversation_helpers import ConversationMessageType


def get_last_reasoning_message(guided_conversation: GuidedConversation) -> str:
    """Given a instance of the GuidedConversation class, this function returns the last reasoning message in the conversation if it exists."""
    messages = guided_conversation.conversation.conversation_messages
    msg = "No previous reasoning message found."
    for message in reversed(messages):
        if message.metadata["type"] == ConversationMessageType.REASONING:
            msg = message.content
            break
    return msg

Let's now reply as the student to the agent's message and see what happens. This is the typical flow of a guided conversation. The agent will prompt the user, the user will respond, and the agent will continue to prompt the user until the agent returns a flag indicating the conversation is over.

In [4]:
user_input = "Ok it's almost summer, I'll try to write a poem about that."

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)

print(response.ai_message)

That's a wonderful choice, David! Summer is a great topic for an acrostic poem. Let's look at an example of an acrostic poem to make sure you understand how to write one. Here's an example using the word 'HAPPY': 

```
Having fun with friends all day,
Awesome games that we all play.
Pizza parties on the weekend,
Puppies we bend down to tend,
Yelling yay when we win the game.
```

Now, you can use your chosen word 'SUMMER' to write your acrostic poem. Write down the letters of 'SUMMER' vertically and think of words or phrases that start with each letter. When you're ready, share your poem with me!


### The Agenda
Usually after the first message from the user, the agent will generate an initial agenda for the conversation. 
Let's examine what it currently looks like. Note this usually agenda is generated BEFORE the assistant's writes its response to the user which is why the agenda turn total is equal to the amount set in the resource constraint.

In [5]:
# Get string representation of the agenda object. We use this same function for formatting into reasoning prompts.
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())

Current agenda:
1. [1 turn] Give an example of an acrostic poem using the word 'HAPPY' - 1 turn
2. [2 turns] Ask David to write his own acrostic poem about summer and submit it - 2 turns
3. [2 turns] Review David’s poem and provide initial feedback - 2 turns
4. [2 turns] Ask David to revise his poem based on the feedback - 2 turns
5. [1 turn] Review revised poem and provide final feedback - 1 turn
6. [1 turn] Verify any inappropriate behavior during the conversation - 1 turn
Total = 9 turns


Now let's give the agent and incomplete poem.

In [6]:
user_input = """Here is my poem so far.
Sun shines alot
U is for ukulele
My friends visit to play basketball
M
E
R"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Great start, David! I see you have 'Sun shines a lot', 'U is for ukulele', and 'My friends visit to play basketball'. Those are excellent lines! Now, let's think of phrases or words that start with the remaining letters M, E, and R. Take your time and when you're ready, share the rest of your poem with me!


The agent tries to guide us to keep writing the poem! 

Let's try to push our luck and have the agent write the rest for us. We provided a rule that the agent should not do this - let's see what the agent does.

In [7]:
user_input = """I got pretty far can you write the rest for me?"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You've made great progress, David! It's really important that you complete the poem yourself so that it's entirely your own work. How about thinking of some fun summer activities or things you love about summer that start with the letters M, E, and R? For example, 'Might be time for ice cream,' 'Evening picnics with family,' or 'Relaxing at the beach.' Give it a try and complete the poem. I believe you can do it!


Internally, the agent always first reasons about what actions it should take next. Let's see what the agent's reasoning was for this turn. This can often help us understand where the agent went wrong.

After we will continue the conversation for a few turns, with the agent guiding us to complete the poem.

In [8]:
# Get the last reasoning message.
print(get_last_reasoning_message(guided_conversation_agent))

### Step-by-Step Reasoning

1. **Acknowledge and Uphold Rules**:
   - David has requested help in writing the rest of his poem, which goes against the guideline that I should not write the poem for the student. It's essential to gently remind David of this and encourage him to finish the poem on his own.

2. **Provide Support and Motivation**:
   - I will provide encouraging words and perhaps some prompts to help him think of words or phrases for the remaining letters.

### Action: Update Agenda

**Items to be completed sequentially**:
1. **Encourage David to complete the poem and provide prompts for the letters M, E, and R** - 1 turn (cumulative total = 4)
2. **Ask David to write his own acrostic poem about summer and submit it** - 1 turn (cumulative total = 5)
3. **Review David’s poem and provide initial feedback** - 2 turns (cumulative total = 7)
4. **Ask David to revise his poem based on the feedback** - 2 turns (cumulative total = 9)
5. **Review revised poem and provide final feed

In [9]:
user_input = "What other things start with e that I could write about?"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Great question, David! Here are a few more things that start with the letter E:
- Evening picnics with family
- Exciting beach games
- Exploring nature trails
- Eating ice cream
Now that you have some ideas for E, remember to think of words or phrases for M and R too. You can do it! Share with me once you're done.


In [10]:
user_input = """Sun shines alot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Fantastic job, David! Here's what you wrote:
```
Sun shines a lot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach
```
I really like how you included different activities and things you enjoy about summer. Let's review your poem and see what you did well and what you might want to improve. Here are some initial thoughts:
1. **Strengths**:
- Great variety of summer activities!
- Good use of imagery with 'Sun shines a lot' and 'Eating lots of popsicles'.
2. **Areas to Improve**:
- Consider making each line a bit more descriptive. For instance, instead of 'U is for ukulele,' you could say 'Ukulele tunes fill the warm air'.
- Try to avoid repetition (e.g., using 'My friends visit to play...' for two different lines).

Take a moment to revise your poem based on this feedback, and then share your revised poem with me!


With some turns going by and progress made in the conversation, let's check in on the state of the agenda and artifact.

If the agent has chosen to update the agenda, we will see the updated agenda. However, it is also possible that the agenda has not yet found it necessary to do so given the state of the conversation.

We should see that the agent has updated the artifact with the current state of the poem since the student has provided it in the previous message.

In [11]:
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current agenda:
1. [1 turn] Review David’s poem and provide initial feedback
2. [2 turns] Ask David to revise his poem based on the feedback
3. [2 turns] Review revised poem and provide final feedback
Total = 5 turns
Current artifact:
{'student_poem': 'Sun shines a lot\nU is for ukulele\nMy friends visit to play basketball\nMy friends also visit to play soccer\nEating lots of popsicles\nRoad trips to the beach', 'initial_feedback': 'Unanswered', 'final_feedback': 'Unanswered', 'inappropriate_behavior': 'Unanswered'}


In [12]:
user_input = """Here are my updates
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the hot beach

But I don't really know what to do for the two my"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Nice work on the updates, David! Your lines 'Sun warms the super fun days' and 'U is for loud ukuleles' are great improvements. 

### Strengths:
- Excellent descriptive language, particularly about the warmth and fun of summer!
- You have effectively included various summer activities and sensory details.

### Areas to Improve:
- To add more variety and avoid repetition in 'My friends visit to play...', consider changing one of the lines to something different. For example, instead of repeating the same phrase, you could use 'Meetups with friends for a game of soccer' or 'Basketball games in the sunny park'.

Give it a try and make those adjustments, then share your final version with me!


In [13]:
user_input = """Ok here is my revised poem

Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Excellent job, David! Here’s what you wrote:

```
Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach
```

You've done a fantastic job in revising your poem. Here are the key improvements and strengths I noticed:

- **Descriptive Language**: Your lines like 'Sun warms the super fun days!' and 'Under clear warm skies my friends play' add vivid details that paint a strong picture of summer.
- **Variety and Imagery**: You've effectively varied your lines and included strong visual imagery and activities, making the poem engaging and fun to read.

Keep up the great work and continue writing! Have a wonderful day!


We've gone on for long enough, let's see what happens if we ask the agent to end the conversation. 

And finally we will print the final state of the artifact after the final update.

In [14]:
user_input = "I'm done for today, goodbye!!"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

No artifact change during final update due to: No tool was called


I will terminate this conversation now. Thank you for your time!


In [15]:
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current artifact:
{'student_poem': 'Sun warms the super fun days!\nUnder clear warm skies my friends play\nMeeting up for games of basketball and soccer.\nMoving butterflies everywhere\nEating lots of chilly popsicles in the sun\nRoad trips to the hot beach', 'initial_feedback': "Great job on revising the poem, David! Your line 'Sun warms the super fun days' and 'U is for loud ukuleles' are wonderful improvements.\n\n### Strengths:\n- Excellent descriptive language, particularly with the warmth and fun themes!\n- You have effectively included various summer activities and sensory details.\n\n### Areas to Improve:\n- To add more variety and avoid repetition for 'My friends visit to play...', consider changing one of the lines to something different. For example, instead of repeating the same phrase, you could use 'Meetups with friends for a game of soccer' or 'Basketball games in the sunny park'.", 'final_feedback': "Fantastic job on your final version, David! You've made some excellent